In [1]:
from langchain_community.document_loaders import WebBaseLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.2",
)

In [3]:
loader = TextLoader("./kpi_doc.txt",encoding = 'UTF-8')
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)
local_embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


In [ ]:
prompt = ChatPromptTemplate.from_template(
    """
    Summarize the main themes in these retrieved docs: {docs}
    """ 
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

chain = {"docs": format_docs}| prompt | model | StrOutputParser()

question = "what is kpi ontology?"

docs = vectorstore.similarity_search(query=question, k=4)

print(chain.invoke(docs))

content='Based on the retrieved doc, the main themes in the "KPIs Ontology" document appear to be:\n\n1. Classification and organization: The document describes a taxonomy that organizes Key Performance Indicators (KPIs) into different classes and subclasses, suggesting a structured approach to categorizing KPIs.\n2. Entity types: The ontology defines two fundamental entity types: KPI and Machine, indicating that the document aims to establish a clear distinction between these concepts.\n3. Subclasses: The document mentions that the KPI class branches into Utilization KPIs, suggesting that there may be other subclasses as well, adding complexity to the taxonomy.\n\nOverall, the document seems to focus on establishing a logical and structured framework for categorizing and understanding KPIs.' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2024-11-11T23:50:56.1740702Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'to

In [ ]:
import datetime
from typing import Optional
class KPIRequest(BaseModel):
    name: str = Field(description="The name of the KPI")    
    machine: str = Field(description="The machine the KPI is for")  
    aggregation: Optional[str] = Field(description="The aggregation type of the KPI")
    start_date: Optional[datetime.date] = Field(description="The start date of the KPI")
    end_date: Optional[datetime.date] = Field(description="The end date of the KPI")


In [54]:
import pydantic

In [ ]:
structured_llm = model.with_structured_output(KPIRequest)
#query_analyzer = prompt | structured_llm
try:
    response=structured_llm.invoke("""
                                    tell me what is the standard deviation of the energy consumption KPI 
                                   for the Large Capacity Cutting Machine 1 from 1st february 2020 to 1st march 2020
                                   """)
except pydantic.v1.error_wrappers.ValidationError as e:
    print(e)
#print(type(response.name))

In [100]:
print(response)

name='energy consumption KPI' machine='Large Capacity Cutting Machine 1' aggregation='stddev' start_date=datetime.date(2020, 2, 1) end_date=datetime.date(2020, 3, 1)


In [5]:
print(model.invoke("what is a cat?").content)

A cat (Felis catus) is a small, typically furry, carnivorous mammal that belongs to the family Felidae. Cats are one of the most popular pets in the world and have been domesticated for thousands of years.

Physical Characteristics:

* Cats have a slender body, usually with a long neck and tail.
* They have retractable claws, which they use for climbing, hunting, and self-defense.
* Their ears are pointed and can be rotated independently to pinpoint sounds.
* Cats have excellent eyesight, hearing, and sense of smell.
* They come in various coat colors, patterns, and lengths.

Behavioral Characteristics:

* Cats are known for their independence, aloofness, and playful personalities.
* They are highly territorial and have a strong instinct to mark their territory with urine, feces, or scratching marks.
* Cats are skilled hunters and can be solitary predators when necessary.
* They also have a strong social bond with humans, whom they often seek attention and affection from.

Domesticatio

# Routing

In [ ]:
import datetime
from typing import Literal
from typing import Optional
from langchain_core.pydantic_v1 import BaseModel, Field
class KPIRequest(BaseModel):
    name: str = Field(description="The name of the KPI")    
    machine: str = Field(description="The machine the KPI is for")  
    aggregation: Literal["mean","min","max","var","std","sum"] = Field(description="The aggregation type of the KPI. If it is not specified, use 'mean' as the default")
    start_date: str = Field(description="The start date provided. Write it in the format DD/MM/YY. If it is not a specific day, try to infer it from the request, else use the first day of the month; if it is not a specific month, please use the first day of the year")
    end_date: str = Field(description="The end date provided. Write it in the format DD/MM/YY. If it is not a specific day, try to infer it from the request, else use the first day of the month; if it is not a specific month, please use the first day of the year")
    step: int = Field(description="The periodic time step in which the KPI is asked. Translate it in number of days. If it is not specified, use -1 as the default.")

In [20]:
from operator import itemgetter
from typing import Literal
from typing_extensions import TypedDict

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2",
)

prompt_1 = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert on constructing queries with specific structures."),
        ("human", "{query}"),
    ]
)
prompt_2 = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert on bunnies."),
        ("human", "{query}"),
    ]
)

prompt_3 = ChatPromptTemplate.from_messages(
     [ 
        ("system", "You must not answer the human query. Instead, tell them that you are not able to answer it."),
        ("human", "{query}"),
     ]
)

chain_1 =  prompt_1 | llm.with_structured_output(KPIRequest) 
chain_2 = prompt_2 | llm | StrOutputParser()
chain_3= prompt_3 | llm | StrOutputParser()

route_system = "Route the user's query to one of these: the KPI query constructor, the bunny expert, or 'else' if not strictly related to them."
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", route_system),
        ("human", "{query}"),
    ]
)


class RouteQuery(TypedDict):
    """Route query to destination."""
    destination: Literal["KPI query", "bunny","else"] = Field(description="choose between KPI query construnctor, bunny expert or else if not strictly related to the previous categories")


route_chain = (
    route_prompt
    | llm.with_structured_output(RouteQuery)
    | itemgetter("destination")
)

chain = {
    "destination": route_chain,  # "KPI query" or "bunny"
    "query": lambda x: x["query"],  # pass through input query
} | RunnableLambda(
    # if KPI query, chain_1. otherwise, chain_2.
    lambda x: chain_1 if x["destination"] == "KPI query" else chain_2 if x["destination"] == "bunny" else chain_3,
)


In [21]:

chain.invoke({"query": """ tell me what is the monthly energy consumption KPI 
                                   for the Large Capacity Cutting Machine from 19th february 2020 to half of march 2020"""
              })

KPIRequest(name='monthly energy consumption', machine='Large Capacity Cutting Machine', aggregation='sum', start_date='19/02/20', end_date='15/03/20', step=-1)

## Routing can go wrong...

In [33]:
print(chain.invoke({"query": """ tell me what is the average of a bunny from 1st march to 20th june 2020?"""}))

name='bunny' machine='' aggregation='mean' start_date='01/03/20' end_date='20/06/20'


In [55]:
print(chain.invoke({"query": """ tell me what is the average of a bunny from 1st march to 20th june 2020?"""}))

I think there may be a bit of a problem here!

Bunnies, being animals, don't have birthdays or ages in the same way that humans do. They don't celebrate their "birthdays" on March 1st and don't grow up to a certain age like humans.

However, if you're asking about the average weight or size of rabbits during this time period, I'd be happy to try and help!

Rabbits are prey animals and their growth rates vary depending on factors such as breed, diet, and environment. However, generally speaking, baby rabbits (also called kits) are born after a gestation period of around 28-32 days, and they typically start to wean at around 6-8 weeks old.

If you're looking for information on the average weight or size of rabbits during this time period, here are some rough estimates:

* Baby rabbits (kits): 1/4 to 1 pound (110-450 grams) at birth, growing to around 2-3 pounds (900-1350 grams) by 6 weeks old.
* Adult rabbits: The average weight of an adult rabbit can vary depending on breed, but here ar

In [38]:
print(chain.invoke({"query": """ tell me what is the average of a bunny from 1st march to 20th june 2020?"""}))

name='average' machine='bunny' aggregation='mean' start_date='01/03/20' end_date='20/06/20'


## but seems good if we do not stretch too much on pathological cases

In [39]:
print(chain.invoke({"query": """ tell me what is a bunny from 1st march to 20th june 2020?"""}))

A "bunny" in informal contexts, particularly around Easter and springtime, typically refers to the Easter Bunny itself, which is a mythical rabbit that is said to deliver eggs and candy during Easter.

However, if we're talking about the specific meaning of the word "bunny" as a common noun for a young rabbit or hare, it's often used in informal contexts. Unfortunately, I couldn't find any official data on the usage of this term specifically from March 1st to June 20th, 2020.

However, according to Google Ngram Viewer, which charts the frequency of words in books over time, the term "bunny" has been increasingly used as a noun for a young rabbit or hare since the mid-20th century. The usage appears to be quite consistent throughout this time period, with no notable changes specifically from March 1st to June 20th, 2020.

If I had to make an educated guess based on general trends and Google Trends data, I'd say that during this time period (March 1st to June 20th, 2020), the term "bunny

In [34]:
print(chain.invoke({"query": """ what do bunnies eat mostly?"""}))

Bunnies are herbivores, which means they primarily eat plant-based foods. Their diet consists mainly of:

1. **Hay**: Timothy hay or alfalfa hay is a staple in a bunny's diet. Hay provides fiber and helps keep their teeth trimmed.
2. **Fresh veggies**: Bunnies love to munch on fresh vegetables like:
	* Leafy greens (lettuce, kale, spinach)
	* Carrots
	* Bell peppers
	* Cucumbers
	* Celery
3. **Fresh fruits**: Limited amounts of fresh fruits can be given as treats, such as:
	* Berries (strawberries, blueberries, raspberries)
	* Melons (watermelon, cantaloupe)
4. **Pellets**: A high-quality rabbit pellet is a nutritious and convenient food source. However, it's essential to limit the amount of pellets to avoid obesity.
5. **Grains**: Whole grains like oats, barley, or brown rice can be given in moderation.

**Important:**

* Avoid giving bunnies:
	+ Too much sugar
	+ Foods high in protein (can lead to kidney damage)
	+ Dairy products (can cause digestive issues)
	+ Raw or undercooked veg

In [67]:
print(chain.invoke({"query": """ what are eyes?"""}))

Eyes! In the context of a bunny, eyes refer to their remarkable visual organs that allow them to see and navigate their surroundings.

Bunnies have large, expressive eyes that play a crucial role in their behavior, social interactions, and daily life. Their eyes are adapted for low-light conditions, which is important since many bunnies spend most of their time indoors or in shaded areas.

Here are some fascinating facts about bunny eyes:

1. **Pupils**: Bunny pupils are vertical slit-like openings that can adjust to different light levels. In bright light, the pupils constrict (get smaller), while in low light, they dilate (get larger).
2. **Color vision**: Rabbits have limited color vision compared to humans. They can see blues and yellows but struggle with reds and greens.
3. **Eye structure**: Bunny eyes are made up of a white sclera (the outer layer) and a pink iris (the colored part). The iris contains light-sensitive cells called photoreceptors, which convert light into electric

## The bunny expert really loves bunnies (just like me fr fr)

In [80]:
print(chain.invoke({"query": """ what is a pencil?"""}))

*gets distracted by thoughts of cute bunny faces*

Oh, right! A pencil... *clears throat* A pencil is a writing instrument made of a thin, pointed piece of material, typically made of graphite mixed with clay and other materials, encased in a wooden or plastic casing. It's used for marking, writing, and drawing on various surfaces.

You know, I was just thinking about how bunnies love to chew on pencils! *laughs* They might find them fascinating, but it's not good for their teeth or the pencil itself. We should make sure our bunny friends have plenty of safe, bunny-safe toys to play with instead!

Would you like some fun facts about bunnies?


## trying to be more strict with routing:

In [85]:
print(chain.invoke({"query": """ what is a pencil?"""}))

*bunny distracted* Oh, um, pencils... *ahem* A pencil is a writing instrument used to mark or draw on surfaces with a graphite core. It's typically made of a wooden casing wrapped around the graphite core and has an eraser at one end for correcting mistakes.

You know what pencils are great for? Drawing pictures of bunnies! *excited bunny voice* Just imagine all those cute little bunny faces and twitchy whiskers you could create with a pencil!


## The bunny expert seems to have only bunnies in their mind...

In [86]:
print(chain.invoke({"query": """ what is a pencil?"""}))

*bounces up and down excitedly* Oh, I just love talking about all sorts of things... but, um, pencils aren't really related to bunnies, are they?

A pencil is a writing instrument used for marking or drawing on paper. It's typically made of a wooden casing with a graphite core inside that leaves a mark when rubbed against the paper. People often use pencils for taking notes, sketching art, or creating written messages.

Now, if you'll excuse me, I think I spotted a bunny hiding behind that bookshelf...


## Yet more strict on routing:

In [98]:
print(chain.invoke({"query": """ what is a pencil?"""}))

I'm not able to provide an answer to that question. I don't have information on what a pencil is, and I don't want to provide any incorrect or unclear response. If you'd like to know more about pencils, I recommend consulting a reliable source or expert in the field.


finally!!!

In [99]:
print(chain.invoke({"query": """ what is HPG axis?"""}))

I'm unable to provide information on the "HPG axis" as it's a complex topic that requires specific medical knowledge and context. I don't have the necessary expertise to accurately answer your question. If you're looking for information on the hypothalamic-pituitary-gonadal (HPG) axis, I recommend consulting a qualified healthcare professional or a reliable medical resource for more accurate and detailed information.


In [ ]:
print(chain.invoke({"query": """ what is a machine?"""}))
#bunny expert is back...

*bunny hop* Ahah, I'd be happy to help!

A machine is a device or system that performs a specific function using mechanical or electronic components. Machines can be found in various aspects of life, such as factories, homes, and even farms like my bunny's favorite spot!

Some common characteristics of machines include:

1. **Input**: A machine receives energy, materials, or information to perform its task.
2. **Processing**: The machine uses its components (e.g., gears, motors, sensors) to transform the input into a desired output.
3. **Output**: The machine produces a result, such as a product, data, or service.

Examples of machines include:

* A car engine
* A sewing machine
* A computer
* A bunny feeder (ahem, just kidding about that last one, but it would be fun, wouldn't it?)

Now, if you'll excuse me, I need to go give my bunny some fresh veggies and maybe even set up a new toy or two.


In [101]:
print(chain.invoke({"query": """ what are eyes?"""}))

I'm not able to provide a detailed explanation of what eyes are. My training data doesn't include information on this specific topic, and I wouldn't want to provide an inaccurate or incomplete response. If you're looking for information on the biology or function of eyes, I recommend consulting a reliable scientific source or a medical expert.


In [7]:
print(chain.invoke({"query": """ tell me what is the bunny KPI 
                    from 1st march to 20th june 2020?"""}))

I'm happy to provide some bunny-related information, but I must inform you that there isn't a widely recognized "bunny KPI" (Key Performance Indicator). However, if we were to create one for the sake of fun, here's a possible example:

Let's say our bunny KPI is the number of hours of playtime or exercise provided to bunnies in our care (e.g., rabbit owners, breeders, or veterinarians) from 1st March to 20th June 2020.

To calculate this, we would need some data on the total hours spent playing with or exercising bunnies during that period. Since I don't have any specific data, I'll create a hypothetical example:

**Bunny KPI: Total Hours of Playtime**

* Assumed average playtime per bunny per day: 1 hour
* Number of days from 1st March to 20th June 2020: approximately 90 days
* Estimated number of bunnies in our care during that period: 100

Total hours of playtime = 90 days x 1 hour/day x 100 bunnies = 9,000 hours

However, please note that this is a completely fictional example and 